In [1]:
# Importando bibliotecas
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import io
import csv
from datetime import *

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 3, Finished, Available, Finished)

In [2]:
# Formatação do tipo Data 

def log():
  return datetime.now().strftime('%Y-%m-%d %H:%M:%S') + ' >>>'


# Incluir variável processamento

dt_proc = datetime.now().strftime('%Y%m%d%H%M%S')
dt_proc

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 4, Finished, Available, Finished)

'20250129210713'

In [3]:
# Leitura  dos arquivos Parquet na pasta Novos
df = spark.read.option('header', 'true').csv("Files/Raw/Fatura/*.csv")

df.createOrReplaceTempView("df")

display(df)

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 359c9069-57bb-4be9-9bbb-16459405eff1)

In [4]:
# Quantidade de Registros

qtd_registros = df.count()
print(log(),f'Quantidade de Registros: {qtd_registros}')

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 6, Finished, Available, Finished)

2025-01-29 21:07:17 >>> Quantidade de Registros: 130000


In [5]:
df.printSchema()

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 7, Finished, Available, Finished)

root
 |-- id_fatura: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- data_emissao: string (nullable = true)
 |-- data_vencimento: string (nullable = true)
 |-- valor_fatura: string (nullable = true)
 |-- valor_pagamento_minimo: string (nullable = true)



In [6]:
df_fatura_formated = spark.sql(f"""
      SELECT
          CAST(Regexp_replace(id_fatura, '-', '') as BIGINT) as id_fatura,
          CAST(id_cliente as BIGINT) as id_cliente,
          "{dt_proc}" as dt_proc,
          SUBSTRING(REPLACE(data_emissao, '-', ''), 1, 6) ref,
          TO_DATE(data_emissao) as data_emissao_fatura,
          TO_DATE(data_vencimento) as data_vencimento_fatura,
          DATE_FORMAT(TO_DATE(data_emissao), 'yyyy') as ano_emissao_fatura,
          CAST(valor_fatura as DECIMAL(10,2)) as valor_fatura,
          CAST(valor_pagamento_minimo as DECIMAL(10,2)) as valor_pagamento_minimo

      FROM df

""")
df_fatura_formated.createOrReplaceTempView('df_fatura_formated')
display(df_fatura_formated)

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b7260dc2-756c-4f8a-a9a5-f2cea657489f)

In [7]:
spark.sql("""
  SELECT
      ref
  FROM df_fatura_formated
  GROUP BY 1
  ORDER BY 1
""").show()

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 9, Finished, Available, Finished)

+------+
|   ref|
+------+
|202301|
|202302|
|202303|
|202304|
|202305|
|202306|
|202307|
|202308|
|202309|
|202310|
|202311|
|202312|
|202401|
+------+



In [8]:
# Salvar arquivo na pasta Trusted

df_fatura_formated.write.mode('append').partitionBy('ref').parquet("Files/Trusted/tb_fatura")

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 10, Finished, Available, Finished)

**Início Carregamento Base Controle**

In [9]:
df_controle_fatura = spark.sql(f"""
            SELECT
                'tb_fatura' as tabela_name,
                "{dt_proc}" as dt_proc,
                {qtd_registros} as qtd_registros

""")
df_controle_fatura.createOrReplaceTempView("df_controle_fatura")

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 11, Finished, Available, Finished)

In [10]:
# Salvar  arquivo de controle de processamento

df_controle_fatura.write.mode('append').parquet("Files/Controle/tb_fatura")

StatementMeta(, de4eff36-b21d-4126-bf5c-75fd54175b52, 12, Finished, Available, Finished)